# Importing Dependencies

### the notebook in a nutshell 
- importing 
software dependencies we need
- load_data (please read its description it supposed to be very easy to deal with it)

###  preprocesssing 
- I stored some important variables, then I used log transformation for long tailed distributions

- iterative imputer just like what `MAAB` Advised

- transormed the categorical features to be from the string type just to avoid some fitting isssues cause by the 
OneHotEncoder.

- droped the `participant_ID` after storing it for stratification

- I scaled quantative data and also FMRI data 

### modeling
- base model is logisitc regression with l2 regulization (fancy name for punishing the model to stop overfitting to noise) 
- wrapped inside a multioutput classifier that just gives it the ability to predict 2 targets simultaneously
### validation 
- 5 splits stratifiedKFold works fine no need for grouping as participants are unique 
- regular stratified  = 0.5901173137436236
- RepeatedStratifiedKFold =0.5865925294336686 with 7 minutese delay

In [1]:
import pandas as pd
import numpy as np          
from sklearn.metrics import f1_score  
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV
from scipy.stats import hmean
from sklearn.multioutput import MultiOutputClassifier
import scipy
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LassoCV
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer #it might not work directly if not try the following code line

from sklearn.model_selection import StratifiedGroupKFold,StratifiedKFold,RepeatedStratifiedKFold
from pathlib import Path
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import f1_score
import warnings
warnings.filterwarnings("ignore")

# Reading Data

In [40]:
"""
just drop the path of your new data directory nothing more than that. 

"""
path = r"C:\Users\Maab\Desktop\ADHD_Kaggle_Competition\Repo\WiDS-Datathon-2025\Data\raw"

def read_data(base_path:str) -> pd.DataFrame :
    path = Path(base_path)
    trc=pd.read_excel(path /'TRAIN_NEW'  / 'TRAIN_CATEGORICAL_METADATA_new.xlsx')
    trq=pd.read_excel(path /'TRAIN_NEW'  / 'TRAIN_QUANTITATIVE_METADATA_new.xlsx')
    trf=pd.read_csv(path   /'TRAIN_NEW'  / 'TRAIN_FUNCTIONAL_CONNECTOME_MATRICES_new_36P_Pearson.csv')
    trs=pd.read_excel(path /'TRAIN_NEW'  / 'TRAINING_SOLUTIONS.xlsx')  
    tsc=pd.read_excel(path /'TEST'      / 'TEST_CATEGORICAL.xlsx')
    tsq=pd.read_excel(path /'TEST'       / 'TEST_QUANTITATIVE_METADATA.xlsx')    
    tsf=pd.read_csv(path   /'TEST'       / 'TEST_FUNCTIONAL_CONNECTOME_MATRICES.csv')    
    sub=pd.read_excel(path / 'SAMPLE_SUBMISSION.xlsx')    
    dic=pd.read_excel(path /'Data Dictionary.xlsx')
    return trc, trq, trf, trs, tsc, tsq, tsf, sub, dic

trc, trq, trf, trs, tsc, tsq, tsf, sub, dic = read_data(base_path=path)

# Data Merging 
cq = pd.merge(trc, trq, on='participant_id', how='left')
feat = pd.merge(cq, trf, on='participant_id', how='left')  
qc = pd.merge(tsc, tsq, on='participant_id', how='left')
train = pd.merge(feat, trs, on='participant_id', how='left') 
test = pd.merge(qc, tsf, on='participant_id', how='left')

# Adding Features before Preprocessing

In [ ]:
# Your coefficients (estimated from your description)
female_coefs = {
    'ColorVision_CV_Score': 2.900,
    'SDQ_SDQ_Emotional_Problems': 2.814,
    'SDQ_SDQ_Prosocial': 2.661,
    'SDQ_SDQ_Internalizing': 1.382,
    'SDQ_SDQ_Conduct_Problems': 1.329,
    'SDQ_SDQ_Generating_Impact': 1.109,
    'APQ_P_APQ_P_PM': 0.642
}

male_coefs = {
    'SDQ_SDQ_Hyperactivity': -3.088,
    'APQ_P_APQ_P_PP': -1.903,
    'SDQ_SDQ_Externalizing': -1.447,
    'APQ_P_APQ_P_CP': -1.380,
    'APQ_P_APQ_P_INV': -0.437,
    'APQ_P_APQ_P_OPD': -0.676,
    'SDQ_SDQ_Peer_Problems': -0.596
}

# Compute weighted sums
train['female_symptom_score'] = sum(train[feat] * coef for feat, coef in female_coefs.items())
train['male_symptom_score'] = sum(train[feat] * coef for feat, coef in male_coefs.items())

test['female_symptom_score'] = sum(test[feat] * coef for feat, coef in female_coefs.items())
test['male_symptom_score'] = sum(test[feat] * coef for feat, coef in male_coefs.items())

# (repeat for test if needed)


In [33]:
train['male_symptom_score'].describe()
# train['male_symptom_score'].dtype
# train[feature].dtype == 'float64'


count    1.213000e+03
mean     1.391211e-16
std      1.000412e+00
min     -2.693816e+00
25%     -7.485035e-01
50%     -8.479617e-03
75%      6.900991e-01
max      2.785302e+00
Name: male_symptom_score, dtype: float64

# Preprocessing

### Setup

In [42]:
# Highlighting important variables. 
# note that I did't deal with the quantative data trq as categorical
# I will use the OneHotEncoder for the categorical data as we have some data trap that I don't think we can use the label encoder for. 

for feature in trc.columns:
    train[feature] = train[feature].astype(object)
train_ids = train['participant_id']
test_ids = test['participant_id'] # I will store them for later usage in grouping in validation why?  I don't want the same user to appear in both train and test. 
num_feats = [feature for feature in train.columns if train[feature].dtype == 'float64']
cat_feats = [feature for feature in train.columns if train[feature].dtype == 'object'] # seperate categorical and numerical features help me reteriving them later easily for preprocessing.
target_cols = ['ADHD_Outcome', 'Sex_F']
groups = train_ids

### Log Transformation

In [43]:
# Features with positive skewness and non-negative values: apply log transformation
# num_feats[:18] Don't transform fMRI data because it has negative values
log_features = [f for f in num_feats[:18] if (train[f] >= 0).all() and scipy.stats.skew(train[f]) > 0.5]  # Only apply to features with significant positive skew

# Apply log transformation for the selected features
for feature in log_features:
    train[feature] = np.log1p(train[feature])  # Apply log(x+1) to handle skewed features
    test[feature] = np.log1p(test[feature])    # Apply log(x+1) to test data as well


### Missing Values

In [44]:
'''
Train has 25 features/columns with missing values: ['PreInt_Demos_Fam_Child_Ethnicity', 'PreInt_Demos_Fam_Child_Race', 'MRI_Track_Scan_Location', 'Barratt_Barratt_P1_Edu', 'Barratt_Barratt_P1_Occ', 'Barratt_Barratt_P2_Edu', 'Barratt_Barratt_P2_Occ', 'EHQ_EHQ_Total', 'ColorVision_CV_Score', 'APQ_P_APQ_P_CP', 'APQ_P_APQ_P_ID', 'APQ_P_APQ_P_INV', 'APQ_P_APQ_P_OPD', 'APQ_P_APQ_P_PM', 'APQ_P_APQ_P_PP', 'SDQ_SDQ_Conduct_Problems', 'SDQ_SDQ_Difficulties_Total', 'SDQ_SDQ_Emotional_Problems', 'SDQ_SDQ_Externalizing', 'SDQ_SDQ_Generating_Impact', 'SDQ_SDQ_Hyperactivity', 'SDQ_SDQ_Internalizing', 'SDQ_SDQ_Peer_Problems', 'SDQ_SDQ_Prosocial', 'MRI_Track_Age_at_Scan']
Test  has 23 features/columns with missing values: ['PreInt_Demos_Fam_Child_Ethnicity', 'PreInt_Demos_Fam_Child_Race', 'Barratt_Barratt_P1_Edu', 'Barratt_Barratt_P1_Occ', 'Barratt_Barratt_P2_Edu', 'Barratt_Barratt_P2_Occ', 'EHQ_EHQ_Total', 'ColorVision_CV_Score', 'APQ_P_APQ_P_CP', 'APQ_P_APQ_P_ID', 'APQ_P_APQ_P_INV', 'APQ_P_APQ_P_OPD', 'APQ_P_APQ_P_PM', 'APQ_P_APQ_P_PP', 'SDQ_SDQ_Conduct_Problems', 'SDQ_SDQ_Difficulties_Total', 'SDQ_SDQ_Emotional_Problems', 'SDQ_SDQ_Externalizing', 'SDQ_SDQ_Generating_Impact', 'SDQ_SDQ_Hyperactivity', 'SDQ_SDQ_Internalizing', 'SDQ_SDQ_Peer_Problems', 'SDQ_SDQ_Prosocial']
fMRI has no missing values
Extra columns in train: ['MRI_Track_Age_at_Scan', 'MRI_Track_Scan_Location']
'''

# Find columns with missing values only
train_missing_features_to_impute = train.columns[train.isnull().any()].tolist() # List of features with missing values in train, only 25 and no missing data in fMRI data
test_missing_features_to_impute = test.columns[test.isnull().any()].tolist() # List of features with missing values in test, only 23 and no missing data in fMRI data

# Initialize the imputer
imputer = IterativeImputer(estimator=LassoCV(random_state=42), max_iter=5, random_state=42)

# Impute in-place
train[train_missing_features_to_impute] = imputer.fit_transform(train[train_missing_features_to_impute])
test[test_missing_features_to_impute] = imputer.fit_transform(test[test_missing_features_to_impute])

### Encode Categorical Features

In [45]:
# Convert all categorical features to strings (to avoid mixed types)
for feature in cat_feats:
    train[feature] = train[feature].astype(str)
    test[feature] = test[feature].astype(str)

# One-Hot Encoding for categorical features
encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)

for feature in cat_feats:
    if feature == 'participant_id':  # Skip participant_id since it's not a feature
        continue

    # Apply OneHotEncoder
    train_encoded = encoder.fit_transform(train[[feature]])
    test_encoded = encoder.transform(test[[feature]])

    # Convert encoded features to DataFrame and append them to the original data
    train_encoded_df = pd.DataFrame(train_encoded, columns=encoder.get_feature_names_out([feature]))
    test_encoded_df = pd.DataFrame(test_encoded, columns=encoder.get_feature_names_out([feature]))

    # Drop the original feature and concatenate the new encoded columns
    train = pd.concat([train.drop(columns=[feature]), train_encoded_df], axis=1)
    test = pd.concat([test.drop(columns=[feature]), test_encoded_df], axis=1)

In [20]:
train.columns[:18]
# 'EHQ_EHQ_Total'
# 'MRI_Track_Age_at_Scan'

Index(['participant_id', 'EHQ_EHQ_Total', 'ColorVision_CV_Score',
       'APQ_P_APQ_P_CP', 'APQ_P_APQ_P_ID', 'APQ_P_APQ_P_INV',
       'APQ_P_APQ_P_OPD', 'APQ_P_APQ_P_PM', 'APQ_P_APQ_P_PP',
       'SDQ_SDQ_Conduct_Problems', 'SDQ_SDQ_Difficulties_Total',
       'SDQ_SDQ_Emotional_Problems', 'SDQ_SDQ_Externalizing',
       'SDQ_SDQ_Generating_Impact', 'SDQ_SDQ_Hyperactivity',
       'SDQ_SDQ_Internalizing', 'SDQ_SDQ_Peer_Problems', 'SDQ_SDQ_Prosocial'],
      dtype='object')

### Drop Irrelevant Columns

In [46]:
for df in (train,test):
    # df.drop(columns=['participant_id'], inplace=True) 
    df.set_index('participant_id', inplace=True)
    df.drop(columns=['EHQ_EHQ_Total'], inplace=True) 
    df.drop(columns=['MRI_Track_Age_at_Scan'], inplace=True) 

### Scale Numerical Features

In [47]:

scaler = StandardScaler()

# Only apply scaling to numerical columns that are not part of the target or categorical features
numerical_features = [col for col in train.columns if col not in target_cols and col not in cat_feats]

# Fit scaler on the numerical features of the train set and transform train and test sets
train[numerical_features] = scaler.fit_transform(train[numerical_features])  # Fit and transform for train set
test[numerical_features] = scaler.transform(test[numerical_features])        # Only transform for test set (avoid data leakage)


# Training and Validation

In [86]:
len(train[target_cols[0]])

1213

In [89]:
train['decision_path']

participant_id
00aIpNTbG5uh        ADHD_Female
00fV0OyyoLfw        ADHD_Female
04X1eiS79T4B        ADHD_Female
05ocQutkURd6          ADHD_Male
06YUNBA9ZRLq        ADHD_Female
                     ...       
zwjJWCRzKhDz        ADHD_Female
zwXD5v17Rx01          ADHD_Male
zWzLCi3NTBTd        ADHD_Female
Zy9GTHDxUbXU    Unclear_No_ADHD
Zye7yYRQohXi    Unclear_No_ADHD
Name: decision_path, Length: 1213, dtype: object

In [106]:
train['female_symptom_score']

participant_id
00aIpNTbG5uh    0.883575
00fV0OyyoLfw    1.511688
04X1eiS79T4B    2.649697
05ocQutkURd6   -1.211181
06YUNBA9ZRLq    1.082316
                  ...   
zwjJWCRzKhDz    1.056965
zwXD5v17Rx01   -1.254879
zWzLCi3NTBTd    0.560478
Zy9GTHDxUbXU   -0.960536
Zye7yYRQohXi   -0.839784
Name: female_symptom_score, Length: 1213, dtype: float64

# Too lazy to label

In [98]:
#Actual Validation and Modeling
import warnings
warnings.filterwarnings("ignore")

from sklearn.model_selection import RepeatedStratifiedKFold, StratifiedKFold
from sklearn.linear_model import LogisticRegressionCV
from sklearn.metrics import roc_auc_score, f1_score
import numpy as np

# Constants
SEED = 42
REPEATS = 5
FOLDS = 5
t_sex = 0.3
t_adhd = 0.4

features_sex = [
    'ColorVision_CV_Score', 'APQ_P_APQ_P_CP',
    'APQ_P_APQ_P_ID', 'APQ_P_APQ_P_INV', 'APQ_P_APQ_P_OPD',
    'APQ_P_APQ_P_PM', 'APQ_P_APQ_P_PP', 'SDQ_SDQ_Conduct_Problems',
    'SDQ_SDQ_Difficulties_Total', 'SDQ_SDQ_Emotional_Problems',
    'SDQ_SDQ_Externalizing', 'SDQ_SDQ_Generating_Impact',
    'SDQ_SDQ_Hyperactivity', 'SDQ_SDQ_Internalizing',
    'SDQ_SDQ_Peer_Problems', 'SDQ_SDQ_Prosocial', 'female_symptom_score', 'male_symptom_score'
]


features_adhd = [
    'ColorVision_CV_Score', 'APQ_P_APQ_P_CP',
    'APQ_P_APQ_P_ID', 'APQ_P_APQ_P_INV', 'APQ_P_APQ_P_OPD',
    'APQ_P_APQ_P_PM', 'APQ_P_APQ_P_PP', 'SDQ_SDQ_Conduct_Problems',
    'SDQ_SDQ_Difficulties_Total', 'SDQ_SDQ_Emotional_Problems',
    'SDQ_SDQ_Externalizing', 'SDQ_SDQ_Generating_Impact',
    'SDQ_SDQ_Hyperactivity', 'SDQ_SDQ_Internalizing',
    'SDQ_SDQ_Peer_Problems', 'SDQ_SDQ_Prosocial', 'female_symptom_score', 'male_symptom_score'
]



female_coefs = {
    'ColorVision_CV_Score': 2.900,
    'SDQ_SDQ_Emotional_Problems': 2.814,
    'SDQ_SDQ_Prosocial': 2.661,
    'SDQ_SDQ_Internalizing': 1.382,
    'SDQ_SDQ_Conduct_Problems': 1.329,
    'SDQ_SDQ_Generating_Impact': 1.109,
    'APQ_P_APQ_P_PM': 0.642
}

male_coefs = {
    'SDQ_SDQ_Hyperactivity': -3.088,
    'APQ_P_APQ_P_PP': -1.903,
    'SDQ_SDQ_Externalizing': -1.447,
    'APQ_P_APQ_P_CP': -1.380,
    'APQ_P_APQ_P_INV': -0.437,
    'APQ_P_APQ_P_OPD': -0.676,
    'SDQ_SDQ_Peer_Problems': -0.596
}


# Features to be interacted with predicted probability of Sex_F = 1
interactions = [
    "APQ_P_APQ_P_INV", "APQ_P_APQ_P_PP", "SDQ_SDQ_Hyperactivity", "SDQ_SDQ_Generating_Impact", 'female_symptom_score', 'male_symptom_score'
]

combinations = train[target_cols[0]].astype(str) + train[target_cols[1]].astype(str)



# Evaluation function


def eval_metrics(y_true, y_pred, weights, label="None", thresh=0.5):
    """Evaluate predictions using F1 Score"""
    
    # Calculate the F1 score with thresholding, using weights for sample weighting
    f1 = f1_score(y_true, (y_pred > thresh).astype(int), sample_weight=weights)
    
    # Print the results
    print(f"{label} -> F1: {f1:.6f}")
    
    # Return the metrics
    return f1


# Initialize lists to store scores and out-of-fold predictions
scores_sex = []
scores_adhd = []
sex_oof = np.zeros(len(train[target_cols[1]]))
adhd_oof = np.zeros(len(train[target_cols[0]]))

# Cross-validation setup
# rskf = RepeatedStratifiedKFold(n_splits=FOLDS, n_repeats=REPEATS, random_state=SEED)
skf = StratifiedKFold(n_splits=FOLDS, shuffle=True, random_state=SEED)

# Logistic RegressionCV parameters
logreg_params = {
    "penalty": "l2",
    "fit_intercept": True,
    "random_state": SEED,
    "solver": "saga",
    "class_weight": "balanced",
    "max_iter": 1000  # Increase this if needed
}

model_sex = LogisticRegression(**logreg_params)
model_adhd = LogisticRegression(**logreg_params)


# Initialize lists to collect coefficients across folds
sex_importances = []
adhd_importances = []


# Start cross-validation loop
for fold, (train_idx, val_idx) in enumerate(skf.split(train, combinations), 1):
    print(f"\n=== Fold {fold} ===")

    # Split data
    X_train, X_val = train.iloc[train_idx].copy(), train.iloc[val_idx].copy()
    X_train = X_train.drop(columns=['decision_path'])
    X_val = X_val.drop(columns=['decision_path'])

    y_train_sex, y_val_sex = train[target_cols[1]].iloc[train_idx], train[target_cols[1]].iloc[val_idx]
    y_train_adhd, y_val_adhd = train[target_cols[0]].iloc[train_idx], train[target_cols[0]].iloc[val_idx]

    # Sample weights
    weights_train = np.where(combinations.iloc[train_idx] == "11", 2, 1)
    weights_val = np.where(combinations.iloc[val_idx] == "11", 2, 1)

    # Train model to predict Sex_F
    model_sex.fit(X_train, y_train_sex, sample_weight=weights_train)
    sex_importances.append(pd.Series(model_sex.coef_[0], index=X_train.columns))
    sex_train_pred = model_sex.predict_proba(X_train)[:, 1]
    sex_val_pred = model_sex.predict_proba(X_val)[:, 1]
    sex_oof[val_idx] += sex_val_pred / REPEATS

    sex_f1 = eval_metrics(y_val_sex, sex_val_pred, weights_val, "Sex_F", thresh=t_sex)
    scores_sex.append((sex_f1))

    # Add predicted sex probabilities
    X_train = X_train.assign(sex_proba=sex_train_pred)
    X_val = X_val.assign(sex_proba=sex_val_pred)

    # Create interaction features
    for col in interactions:
        X_train[f"I_{col}"] = X_train[col] * X_train["sex_proba"]
        X_val[f"I_{col}"] = X_val[col] * X_val["sex_proba"]

    # Train model to predict ADHD outcome
    model_adhd.fit(X_train[features_adhd], y_train_adhd, sample_weight=weights_train)
    adhd_importances.append(pd.Series(model_adhd.coef_[0], index=X_train[features_adhd].columns))
    adhd_val_pred = model_adhd.predict_proba(X_val[features_adhd])[:, 1]
    adhd_oof[val_idx] += adhd_val_pred / REPEATS

    adhd_f1 = eval_metrics(y_val_adhd, adhd_val_pred, weights_val, "Outcome ADHD", thresh=t_adhd)
    scores_adhd.append((adhd_f1))


# Aggregate coefficients (mean across folds)
mean_sex_importance = pd.concat(sex_importances, axis=1).mean(axis=1).sort_values(key=abs, ascending=False)
mean_adhd_importance = pd.concat(adhd_importances, axis=1).mean(axis=1).sort_values(key=abs, ascending=False)

# Print final results
print(f"\n=== CV Results ===")
print(f"Sex Mean F1: {np.mean([f for _, f in scores_sex]):.4f}")
print(f"ADHD Mean F1: {np.mean([f for _, f in scores_adhd]):.4f}")
print("\n=== Feature Importance for Sex Prediction ===")
print(mean_sex_importance)
print("\n=== Feature Importance for ADHD Prediction ===")
print(mean_adhd_importance)





=== Fold 1 ===
Sex_F -> F1: 0.7021, ROC-AUC: 0.7781
Outcome ADHD -> F1: 0.8889, ROC-AUC: 0.8723

=== Fold 2 ===
Sex_F -> F1: 0.7200, ROC-AUC: 0.8197
Outcome ADHD -> F1: 0.8710, ROC-AUC: 0.8294

=== Fold 3 ===
Sex_F -> F1: 0.7089, ROC-AUC: 0.7969
Outcome ADHD -> F1: 0.8472, ROC-AUC: 0.8280

=== Fold 4 ===
Sex_F -> F1: 0.7163, ROC-AUC: 0.8001
Outcome ADHD -> F1: 0.8710, ROC-AUC: 0.8487

=== Fold 5 ===
Sex_F -> F1: 0.6893, ROC-AUC: 0.8131
Outcome ADHD -> F1: 0.8909, ROC-AUC: 0.8045

=== CV Results ===
Sex Mean roc auc Score: 0.7073
Sex Mean F1: 0.8016
ADHD Mean roc auc Score: 0.8738
ADHD Mean F1: 0.8366

=== Feature Importance for Sex Prediction ===
ADHD_Outcome           -1.521808e-01
Sex_F                   1.119452e-01
158throw_191thcolumn   -5.062167e-02
male_symptom_score      4.925463e-02
female_symptom_score    3.994496e-02
                            ...     
23throw_78thcolumn      1.751886e-06
8throw_123thcolumn      8.788275e-07
48throw_113thcolumn    -2.953176e-08
34throw_78t

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score
from scipy.stats import hmean
import numpy as np
import pandas as pd

# Setup
target_cols = ['ADHD_Outcome', 'Sex_F']
features = [col for col in train.columns if col not in target_cols and col != 'decision_path']
n_splits = 5
cv = StratifiedKFold(n_splits=n_splits)
base_model_params = dict(random_state=7, penalty='l2', C=0.02, class_weight='balanced')

# Initialize
F1s = []
stds = []
importances = {col: [] for col in target_cols}
thresholds_per_fold = {col: [] for col in target_cols}

# Function to find optimal threshold
def tune_threshold(y_true, y_proba, weights):
    best_thresh = 0.5
    best_f1 = -1
    for thresh in np.linspace(0.1, 0.9, 81):
        preds = (y_proba > thresh).astype(int)
        score = f1_score(y_true, preds, sample_weight=weights)
        if score > best_f1:
            best_f1 = score
            best_thresh = thresh
    return best_thresh, best_f1

# Validation function
def validate(trainset, testset, target_cols):
    f1_scores = []
    fold_coefs = {}
    thresholds = {}

    for col in target_cols:
        weights = ((trainset['Sex_F'] == 1) & (trainset['ADHD_Outcome'] == 1)).astype(int) + 1
        model = LogisticRegression(**base_model_params)
        model.fit(trainset[features], trainset[col], sample_weight=weights)
        proba = model.predict_proba(testset[features])[:, 1]

        valid_idx = testset[col].notna()
        y_true = testset.loc[valid_idx, col]
        y_proba = proba[valid_idx]
        w = ((testset.loc[valid_idx, 'Sex_F'] == 1) & (testset.loc[valid_idx, 'ADHD_Outcome'] == 1)).astype(int) + 1

        # Tune threshold
        best_thresh, best_f1 = tune_threshold(y_true, y_proba, w)
        thresholds[col] = best_thresh
        f1_scores.append(best_f1)

        # Store coefficients
        fold_coefs[col] = pd.Series(model.coef_[0], index=features)

    return f1_scores, fold_coefs, thresholds

# Cross-validation loop
for fold_number, (train_index, test_index) in enumerate(cv.split(train[features], train[target_cols[0]]), 1):
    train_v, test_v = train.iloc[train_index], train.iloc[test_index]
    train_v_full, test_v_full = train.iloc[train_index], train.iloc[test_index]

    # # Drop 'decision_path' only when used for modeling
    train_v = train_v_full.drop(columns=['decision_path'])
    test_v = test_v_full.drop(columns=['decision_path'])


    f1_scores, fold_coefs, fold_thresholds = validate(train_v, test_v, target_cols)

    print(f"\n==== Fold {fold_number} ===")
    for i, col in enumerate(target_cols):
        print(f"{col} -> Best F1: {f1_scores[i]:.4f} at threshold {fold_thresholds[col]:.2f}")
        importances[col].append(fold_coefs[col])
        thresholds_per_fold[col].append(fold_thresholds[col])


    stds.append(test_v[target_cols].std())
    F1s.append(f1_scores)

# Post-CV Summary
F1s = np.array(F1s)
mean_f1_scores = F1s.mean(axis=0)
f1_stds = F1s.std(axis=0)

print("\n==== Overall Results ===")
print("Mean F1-scores:")
for i, target in enumerate(target_cols):
    print(f"    {target} -> F1: {mean_f1_scores[i]:.5f}")

print("\nF1-score stds:")
for i, target in enumerate(target_cols):
    print(f"    {target} -> Std: {f1_stds[i]:.5f}")

print("\nStandard deviations of test sets:")
for std in stds:
    print(f"    {std.to_dict()}")

# Final Harmonic Mean
print("\nScore mean (ADHD, Sex):", np.mean(F1s, axis=0))
score = hmean(F1s, axis=0)
score = hmean(score, axis=0)
print("\nFinal Overall Harmonic Mean (Mimic Leaderboard):", score)

# Average Feature Importances
print("\n==== Average Feature Importances Across Folds ===")
for col in target_cols:
    print(f"\n{col}:")
    avg_importance = pd.concat(importances[col], axis=1).mean(axis=1)
    print(avg_importance.sort_values(key=abs, ascending=False).round(4))

# Average thresholds
print("\n==== Average Optimal Thresholds ===")
for col in target_cols:
    avg_thresh = np.mean(thresholds_per_fold[col])
    print(f"{col} -> Avg Threshold: {avg_thresh:.6f}")



==== Fold 1 ===
ADHD_Outcome -> Best F1: 0.8423 at threshold 0.11

ADHD_Outcome Feature Importances (Fold 1):
SDQ_SDQ_Hyperactivity          0.1520
SDQ_SDQ_Externalizing          0.1334
male_symptom_score            -0.1297
SDQ_SDQ_Difficulties_Total     0.1225
SDQ_SDQ_Generating_Impact      0.1095
                                ...  
130throw_156thcolumn          -0.0000
40throw_65thcolumn             0.0000
39throw_127thcolumn            0.0000
33throw_148thcolumn            0.0000
Barratt_Barratt_P2_Occ_30.0    0.0000
Length: 20546, dtype: float64
Sex_F -> Best F1: 0.7338 at threshold 0.12

Sex_F Feature Importances (Fold 1):
male_symptom_score       0.0401
158throw_191thcolumn    -0.0343
SDQ_SDQ_Hyperactivity   -0.0320
82throw_84thcolumn      -0.0302
SDQ_SDQ_Externalizing   -0.0291
                          ...  
55throw_97thcolumn      -0.0000
25throw_77thcolumn       0.0000
25throw_91thcolumn      -0.0000
159throw_199thcolumn    -0.0000
157throw_185thcolumn    -0.0000
Length: 2

In [112]:
# Check the data types of all columns
print("Column data types:\n", train.dtypes)

# Identify columns that are categorical (either 'object' or 'category' types)
categorical_columns = train.select_dtypes(include=['object', 'category']).columns
print("\nCategorical Columns:", categorical_columns)

# Optionally, you can also check for any non-numeric columns (besides 'object' or 'category')
non_numeric_columns = train.select_dtypes(exclude=['number']).columns
print("\nNon-Numeric Columns:", non_numeric_columns)


Column data types:
 ColorVision_CV_Score                        float64
APQ_P_APQ_P_CP                              float64
APQ_P_APQ_P_ID                              float64
APQ_P_APQ_P_INV                             float64
APQ_P_APQ_P_OPD                             float64
                                             ...   
Barratt_Barratt_P2_Occ_40.0                 float64
Barratt_Barratt_P2_Occ_45.0                 float64
Barratt_Barratt_P2_Occ_5.0                  float64
Barratt_Barratt_P2_Occ_8.099594997430632    float64
decision_path                                object
Length: 20549, dtype: object

Categorical Columns: Index(['decision_path'], dtype='object')

Non-Numeric Columns: Index(['decision_path'], dtype='object')


### Final Training and Submission

In [97]:
# Drop decision_path if present
X_full = train.drop(columns=['decision_path']).copy()
y_full_sex = train[target_cols[1]]
y_full_adhd = train[target_cols[0]]

# Train final model on full data for Sex prediction
final_model_sex = LogisticRegression(**logreg_params)
final_model_sex.fit(X_full[features_sex], y_full_sex)

# Predict sex probabilities on test set
test_X = test.copy()
sex_test_proba = final_model_sex.predict_proba(test_X[features_sex])[:, 1]

# Add interaction features
test_X = test_X.assign(sex_proba=sex_test_proba)
for col in interactions:
    test_X[f"I_{col}"] = test_X[col] * test_X["sex_proba"]

# Train final ADHD model using full training data + interaction features
X_full = X_full.assign(sex_proba=final_model_sex.predict_proba(X_full[features_sex])[:, 1])
for col in interactions:
    X_full[f"I_{col}"] = X_full[col] * X_full["sex_proba"]

final_model_adhd = LogisticRegression(**logreg_params)
sample_weights_final = np.where(combinations == "11", 2, 1)
final_model_adhd.fit(X_full[features_adhd], y_full_adhd, sample_weight=sample_weights_final)

# Predict ADHD probabilities on test set
adhd_test_proba = final_model_adhd.predict_proba(test_X[features_adhd])[:, 1]

# Threshold probabilities to get final binary predictions
test_pred_sex = (sex_test_proba > t_sex).astype(int)
test_pred_adhd = (adhd_test_proba > t_adhd).astype(int)

# Create submission DataFrame
submission = pd.DataFrame({
    "Id": test.index,
    "Sex_F": test_pred_sex,
    "ADHD_Outcome": test_pred_adhd
})

# Save to CSV
submission.to_csv("submissionLogRegWithNewFeats.csv", index=False)
print("✅ Submission saved as 'submissionLogRegWithNewFeats.csv'")


✅ Submission saved as 'submissionLogRegWithNewFeats.csv'


=== Fold 1 ===
Sex_F -> F1: 0.7021, ROC-AUC: 0.7781
Outcome ADHD -> F1: 0.8889, ROC-AUC: 0.8723

=== Fold 2 ===
Sex_F -> F1: 0.7200, ROC-AUC: 0.8197
Outcome ADHD -> F1: 0.8710, ROC-AUC: 0.8294

=== Fold 3 ===
Sex_F -> F1: 0.7089, ROC-AUC: 0.7969
Outcome ADHD -> F1: 0.8472, ROC-AUC: 0.8280

=== Fold 4 ===
Sex_F -> F1: 0.7163, ROC-AUC: 0.8001
Outcome ADHD -> F1: 0.8710, ROC-AUC: 0.8487

=== Fold 5 ===
Sex_F -> F1: 0.6893, ROC-AUC: 0.8131
Outcome ADHD -> F1: 0.8909, ROC-AUC: 0.8045

=== CV Results ===
Sex Mean roc auc Score: 0.7073
Sex Mean F1: 0.8016
ADHD Mean roc auc Score: 0.8738
ADHD Mean F1: 0.8366

In [79]:
FEMALE_THRESHOLD = 0.3
MALE_THRESHOLD = 0.7 

In [83]:
def find_best_threshold(data, score_col, sex_filter, label, target_col='ADHD_Outcome'):
    best_thresh = 0
    best_f1 = 0
    thresholds = np.linspace(0.0, 1.0, 101)

    # Filter by sex
    subset = data[sex_filter].copy()

    for thresh in thresholds:
        preds = (subset[score_col] > thresh).astype(int)
        f1 = f1_score(subset[target_col], preds)
        if f1 > best_f1:
            best_f1 = f1
            best_thresh = thresh            

    print(f"Best threshold for {label}: {best_thresh:.2f} with F1: {best_f1:.4f}")
    return best_thresh


# Find best thresholds
FEMALE_THRESHOLD = find_best_threshold(train, 'female_symptom_score', train['Sex_F'] == 1, 'ADHD_Female')
MALE_THRESHOLD = find_best_threshold(train, 'male_symptom_score', train['Sex_F'] == 0, 'ADHD_Male')      


Best threshold for ADHD_Female: 0.00 with F1: 0.6783
Best threshold for ADHD_Male: 0.00 with F1: 0.3987


In [80]:
def route_decision(row):
    fem_score = row['female_symptom_score']
    male_score = row['male_symptom_score']

    # if male_score > MALE_THRESHOLD:
    #     if fem_score > FEMALE_THRESHOLD:
    #         return "ADHD_Female"
    # elif fem_score > FEMALE_THRESHOLD:
    #     return "ADHD_Female"
    # elif male_score > MALE_THRESHOLD:
    #     return "ADHD_Male"
    # else:
    #     return "Unclear_No_ADHD"
    
    if fem_score > FEMALE_THRESHOLD:
        return "ADHD_Female"
    elif male_score > MALE_THRESHOLD:
        return "ADHD_Male"
    else:
        return "Unclear_No_ADHD"
    
train['decision_path'] = train.apply(route_decision, axis=1)


In [ ]:
from sklearn.linear_model import LogisticRegressionCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
import lightgbm as lgb

# Example feature groupings (update with your actual feature names)
symptom_features = ['ColorVision_CV_Score', 'SDQ_SDQ_Emotional_Problems', 'SDQ_SDQ_Prosocial', 
                    'SDQ_SDQ_Internalizing', 'SDQ_SDQ_Conduct_Problems', 'SDQ_SDQ_Generating_Impact',
                    'APQ_P_APQ_P_PM', 'SDQ_SDQ_Hyperactivity', 'APQ_P_APQ_P_PP', 'SDQ_SDQ_Externalizing', 
                    'APQ_P_APQ_P_CP', 'APQ_P_APQ_P_INV', 'APQ_P_APQ_P_OPD', 'SDQ_SDQ_Peer_Problems'] 


fmri_features = [feature for feature in train.columns if 'throw' in feature]  

# train['participant_id'] = train['participant_id']


# Create datasets based on decision_path
adhd_male_female = train[train['decision_path'].isin(["ADHD_Female", "ADHD_Male"])]
unclear_no_adhd = train[train['decision_path'] == "Unclear_No_ADHD"]

# Create feature/target sets
X_symptoms = adhd_male_female[symptom_features]
y_symptoms_sex = adhd_male_female['Sex_F']   # assuming you have this
y_symptoms_adhd = adhd_male_female['ADHD_Outcome']  # assuming you have this

X_fmri = unclear_no_adhd[fmri_features]
y_fmri_sex = unclear_no_adhd['Sex_F']

# ------------------------------
# Models for symptom features path (ADHD_Male/Female)
# ------------------------------

SEED = 42
logreg_params = {
    "penalty": "l2",
    "fit_intercept": True,
    "random_state": SEED,
    "solver": "saga",
    "class_weight": "balanced",
    "max_iter": 1000  # Increase this if needed
}

# Main model: Logistic Regression
symptom_model_lr = LogisticRegression(**logreg_params)
symptom_model_lr.fit(X_symptoms, y_symptoms_sex)

# Other models
symptom_model_lgb = lgb.LGBMClassifier(verbose=-1)
symptom_model_lgb.fit(X_symptoms, y_symptoms_sex)

# symptom_model_rf = RandomForestClassifier()
# symptom_model_rf.fit(X_symptoms, y_symptoms_sex)

# symptom_model_svm = SVC(probability=True)
# symptom_model_svm.fit(X_symptoms, y_symptoms_sex)



# ------------------------------
# Models for fMRI path (Unclear/No ADHD)
# ------------------------------

fmri_model_lr = LogisticRegression(**logreg_params)
fmri_model_lr.fit(X_fmri, y_fmri_sex)

# (Optional) Other models for fMRI too
fmri_model_lgb = lgb.LGBMClassifier(verbose=-1)
fmri_model_lgb.fit(X_fmri, y_fmri_sex)

# fmri_model_rf = RandomForestClassifier()
# fmri_model_rf.fit(X_fmri, y_fmri_sex)

# fmri_model_svm = SVC(probability=True)
# fmri_model_svm.fit(X_fmri, y_fmri_sex)



# ------------------------------
# Now you have separate pipelines trained!
# ------------------------------


### Validation

In [81]:
# Wake up uncomment, run to see affect of new threshold. make submission with threshold with best cv
# Main model: Logistic Regression
symptom_model_lr = LogisticRegression(**logreg_params)

# Constants
n_splits = 5
SEED = 42
cv = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=SEED)

# Validation function for symptom features (predicting Sex_F and ADHD_Outcome)
def validate_symptom(trainset, testset, feature_cols):
    target_cols = ['Sex_F', 'ADHD_Outcome']
    f1_scores = []

    for target in target_cols:
        logreg_params = {
            "penalty": "l2",
            "fit_intercept": True,
            "random_state": SEED,
            "solver": "saga",
            "class_weight": "balanced",
            "max_iter": 1000  # Increase this if needed
        }
        model = LogisticRegression(**logreg_params)

        # Define weights only for ADHD
        if target == 'ADHD_Outcome':
            weights = ((trainset['Sex_F'] == 1) & (trainset['ADHD_Outcome'] == 1)).astype(int) + 1
            model.fit(trainset[feature_cols], trainset[target], sample_weight=weights)
        else:
            model.fit(trainset[feature_cols], trainset[target])

        preds = model.predict(testset[feature_cols])

        valid_idx = testset[target].notna()
        true_vals = testset.loc[valid_idx, target]
        valid_preds = preds[valid_idx]

        score = f1_score(true_vals, valid_preds)
        f1_scores.append(score)

    return f1_scores

# Validation function for fMRI features (predicting Sex_F only)
def validate_fmri(trainset, testset, feature_cols):
    target = 'Sex_F'
    logreg_params = {
            "penalty": "l2",
            
            "fit_intercept": True,
            "random_state": SEED,
            "solver": "saga",
            "class_weight": "balanced",
            "max_iter": 1000  # Increase this if needed
    }
    model = LogisticRegression(**logreg_params)

    model.fit(trainset[feature_cols], trainset[target])
    preds = model.predict(testset[feature_cols])

    valid_idx = testset[target].notna()
    true_vals = testset.loc[valid_idx, target]
    valid_preds = preds[valid_idx]

    score = f1_score(true_vals, valid_preds)
    return score


# =========================
# Symptom features validation
# =========================

print("\n--- Validating Symptom Features ---\n")

stds_symptom = []
F1s_symptom = []

for fold_number, (train_index, test_index) in enumerate(cv.split(adhd_male_female[symptom_features], adhd_male_female['ADHD_Outcome']), 1):
    train_v = adhd_male_female.iloc[train_index]
    test_v = adhd_male_female.iloc[test_index]

    f1_scores = validate_symptom(train_v, test_v, symptom_features)

    print(f"\n==== Fold {fold_number} ===")
    print(f"Sex_F -> F1: {f1_scores[0]:.4f}")
    print(f"ADHD_Outcome -> F1: {f1_scores[1]:.4f}")

    stds_symptom.append(test_v[['Sex_F', 'ADHD_Outcome']].std())
    F1s_symptom.append(f1_scores)

F1s_symptom = np.array(F1s_symptom)

# Results
mean_f1_scores_symptom = F1s_symptom.mean(axis=0)
print("\n==== Overall Symptom Results ===")
print("Mean F1-scores:")
print(f"    Sex_F -> F1: {mean_f1_scores_symptom[0]:.4f}")
print(f"    ADHD_Outcome -> F1: {mean_f1_scores_symptom[1]:.4f}")

f1_stds_symptom = F1s_symptom.std(axis=0)
print("\nF1-score stds:")
print(f"    Sex_F -> Std: {f1_stds_symptom[0]:.4f}")
print(f"    ADHD_Outcome -> Std: {f1_stds_symptom[1]:.4f}")

print("\nStandard deviations of test sets:")
for std in stds_symptom:
    print(f"    {std.to_dict()}")

print("\nScore mean (Sex_F, ADHD_Outcome):", np.mean(F1s_symptom, axis=0))
score_symptom = hmean(F1s_symptom, axis=0)
score_symptom = hmean(score_symptom, axis=0)
print("\nFinal Overall Harmonic Mean (Symptom Features):", score_symptom)


# =========================
# fMRI features validation
# =========================

print("\n--- Validating fMRI Features ---\n")

stds_fmri = []
F1s_fmri = []

for fold_number, (train_index, test_index) in enumerate(cv.split(unclear_no_adhd[fmri_features], unclear_no_adhd['Sex_F']), 1):
    train_v = unclear_no_adhd.iloc[train_index]
    test_v = unclear_no_adhd.iloc[test_index]

    f1_score_fmri = validate_fmri(train_v, test_v, fmri_features)

    print(f"\n==== Fold {fold_number} ===")
    print(f"Sex_F -> F1: {f1_score_fmri:.4f}")

    stds_fmri.append(test_v['Sex_F'].std())
    F1s_fmri.append(f1_score_fmri)

F1s_fmri = np.array(F1s_fmri)

# Results
mean_f1_score_fmri = F1s_fmri.mean()
print("\n==== Overall fMRI Results ===")
print(f"Mean F1-score: {mean_f1_score_fmri:.4f}")

f1_std_fmri = F1s_fmri.std()
print(f"F1-score Std: {f1_std_fmri:.4f}")

print("\nStandard deviations of test sets:")
for std in stds_fmri:
    print(f"    {std}")

print("\nScore mean (Sex_F):", np.mean(F1s_fmri))
score_fmri = hmean(F1s_fmri)
print("\nFinal Overall Harmonic Mean (fMRI Features):", score_fmri)



--- Validating Symptom Features ---


==== Fold 1 ===
Sex_F -> F1: 0.5455
ADHD_Outcome -> F1: 0.8462

==== Fold 2 ===
Sex_F -> F1: 0.5079
ADHD_Outcome -> F1: 0.8837

==== Fold 3 ===
Sex_F -> F1: 0.6479
ADHD_Outcome -> F1: 0.8594

==== Fold 4 ===
Sex_F -> F1: 0.5823
ADHD_Outcome -> F1: 0.8455

==== Fold 5 ===
Sex_F -> F1: 0.5352
ADHD_Outcome -> F1: 0.8226

==== Overall Symptom Results ===
Mean F1-scores:
    Sex_F -> F1: 0.5638
    ADHD_Outcome -> F1: 0.8515

F1-score stds:
    Sex_F -> Std: 0.0483
    ADHD_Outcome -> Std: 0.0200

Standard deviations of test sets:
    {'Sex_F': 0.4859126579037751, 'ADHD_Outcome': 0.3804643037391985}
    {'Sex_F': 0.4648111258522642, 'ADHD_Outcome': 0.3804643037391985}
    {'Sex_F': 0.49659333289081214, 'ADHD_Outcome': 0.3804643037391985}
    {'Sex_F': 0.5029973450366553, 'ADHD_Outcome': 0.3712363878934856}
    {'Sex_F': 0.49025309919043125, 'ADHD_Outcome': 0.3823643964146893}

Score mean (Sex_F, ADHD_Outcome): [0.56375363 0.85147178]

Final Overall Har

In [ ]:
"""
sex threshold = 0.5
--- Validating Symptom Features ---


==== Fold 1 ===
Sex_F -> F1: 0.6429
ADHD_Outcome -> F1: 0.8947

==== Fold 2 ===
Sex_F -> F1: 0.5517
ADHD_Outcome -> F1: 0.8411

==== Fold 3 ===
Sex_F -> F1: 0.4746
ADHD_Outcome -> F1: 0.8491

==== Fold 4 ===
Sex_F -> F1: 0.6571
ADHD_Outcome -> F1: 0.7959

==== Fold 5 ===
Sex_F -> F1: 0.5200
ADHD_Outcome -> F1: 0.8462

==== Overall Symptom Results ===
Mean F1-scores:
    Sex_F -> F1: 0.5693
    ADHD_Outcome -> F1: 0.8454

F1-score stds:
    Sex_F -> Std: 0.0705
    ADHD_Outcome -> Std: 0.0314

Standard deviations of test sets:
    {'Sex_F': 0.4929262571578128, 'ADHD_Outcome': 0.38405442985621985}
    {'Sex_F': 0.4909861406609373, 'ADHD_Outcome': 0.37323381758230945}
    {'Sex_F': 0.4830927051164929, 'ADHD_Outcome': 0.3732338175823095}
    {'Sex_F': 0.5037175233040658, 'ADHD_Outcome': 0.37323381758230945}
    {'Sex_F': 0.4872875106609372, 'ADHD_Outcome': 0.37323381758230945}

Score mean (Sex_F, ADHD_Outcome): [0.56926008 0.84539743]

Final Overall Harmonic Mean (Symptom Features): 0.6737383760312459

--- Validating fMRI Features ---


==== Fold 1 ===
Sex_F -> F1: 0.5581
"""





"""
Threshold for sex = 0.3
--- Validating Symptom Features ---


==== Fold 1 ===
Sex_F -> F1: 0.5455
ADHD_Outcome -> F1: 0.8462

==== Fold 2 ===
Sex_F -> F1: 0.5079
ADHD_Outcome -> F1: 0.8837

==== Fold 3 ===
Sex_F -> F1: 0.6479
ADHD_Outcome -> F1: 0.8594

==== Fold 4 ===
Sex_F -> F1: 0.5823
ADHD_Outcome -> F1: 0.8455

==== Fold 5 ===
Sex_F -> F1: 0.5352
ADHD_Outcome -> F1: 0.8226

==== Overall Symptom Results ===
Mean F1-scores:
    Sex_F -> F1: 0.5638
    ADHD_Outcome -> F1: 0.8515

F1-score stds:
    Sex_F -> Std: 0.0483
    ADHD_Outcome -> Std: 0.0200

Standard deviations of test sets:
    {'Sex_F': 0.4859126579037751, 'ADHD_Outcome': 0.3804643037391985}
    {'Sex_F': 0.4648111258522642, 'ADHD_Outcome': 0.3804643037391985}
    {'Sex_F': 0.49659333289081214, 'ADHD_Outcome': 0.3804643037391985}
    {'Sex_F': 0.5029973450366553, 'ADHD_Outcome': 0.3712363878934856}
    {'Sex_F': 0.49025309919043125, 'ADHD_Outcome': 0.3823643964146893}

Score mean (Sex_F, ADHD_Outcome): [0.56375363 0.85147178]

Final Overall Harmonic Mean (Symptom Features): 0.6753660100375886

--- Validating fMRI Features ---
"""



"""
FEMALE_THRESHOLD = 0.3
MALE_THRESHOLD = 0.6

--- Validating Symptom Features ---


==== Fold 1 ===
Sex_F -> F1: 0.5455
ADHD_Outcome -> F1: 0.8462

==== Fold 2 ===
Sex_F -> F1: 0.5079
ADHD_Outcome -> F1: 0.8837

==== Fold 3 ===
Sex_F -> F1: 0.6479
ADHD_Outcome -> F1: 0.8594

==== Fold 4 ===
Sex_F -> F1: 0.5823
ADHD_Outcome -> F1: 0.8455

==== Fold 5 ===
Sex_F -> F1: 0.5352
ADHD_Outcome -> F1: 0.8226

==== Overall Symptom Results ===
Mean F1-scores:
    Sex_F -> F1: 0.5638
    ADHD_Outcome -> F1: 0.8515

F1-score stds:
    Sex_F -> Std: 0.0483
    ADHD_Outcome -> Std: 0.0200

Standard deviations of test sets:
    {'Sex_F': 0.4859126579037751, 'ADHD_Outcome': 0.3804643037391985}
    {'Sex_F': 0.4648111258522642, 'ADHD_Outcome': 0.3804643037391985}
    {'Sex_F': 0.49659333289081214, 'ADHD_Outcome': 0.3804643037391985}
    {'Sex_F': 0.5029973450366553, 'ADHD_Outcome': 0.3712363878934856}
    {'Sex_F': 0.49025309919043125, 'ADHD_Outcome': 0.3823643964146893}

Score mean (Sex_F, ADHD_Outcome): [0.56375363 0.85147178]

Final Overall Harmonic Mean (Symptom Features): 0.6753660100375886

--- Validating fMRI Features ---
"""





"""
FEMALE_THRESHOLD = 0.3
MALE_THRESHOLD = 0.7

--- Validating Symptom Features ---


==== Fold 1 ===
Sex_F -> F1: 0.5455
ADHD_Outcome -> F1: 0.8462

==== Fold 2 ===
Sex_F -> F1: 0.5079
ADHD_Outcome -> F1: 0.8837

==== Fold 3 ===
Sex_F -> F1: 0.6479
ADHD_Outcome -> F1: 0.8594

==== Fold 4 ===
Sex_F -> F1: 0.5823
ADHD_Outcome -> F1: 0.8455

==== Fold 5 ===
Sex_F -> F1: 0.5352
ADHD_Outcome -> F1: 0.8226

==== Overall Symptom Results ===
Mean F1-scores:
    Sex_F -> F1: 0.5638
    ADHD_Outcome -> F1: 0.8515

F1-score stds:
    Sex_F -> Std: 0.0483
    ADHD_Outcome -> Std: 0.0200

Standard deviations of test sets:
    {'Sex_F': 0.48591265790377497, 'ADHD_Outcome': 0.3804643037391984}
    {'Sex_F': 0.4648111258522642, 'ADHD_Outcome': 0.3804643037391984}
    {'Sex_F': 0.4965933328908123, 'ADHD_Outcome': 0.3804643037391984}
    {'Sex_F': 0.5029973450366553, 'ADHD_Outcome': 0.3712363878934856}
    {'Sex_F': 0.49025309919043114, 'ADHD_Outcome': 0.3823643964146896}

Score mean (Sex_F, ADHD_Outcome): [0.56375363 0.85147178]

Final Overall Harmonic Mean (Symptom Features): 0.6753660100375886

--- Validating fMRI Features ---


==== Fold 1 ===
Sex_F -> F1: 0.4632
"""

### Submission

In [82]:
# ------------- Final Training on All Data -------------

# Final model parameters
logreg_params = {
    "penalty": "l2",
    "fit_intercept": True,
    "random_state": SEED,
    "solver": "saga",
    "class_weight": "balanced",
    "max_iter": 1000
}

# SYMPTOM-BASED MODELS
model_sex_symptom = LogisticRegression(**logreg_params)
model_adhd_symptom = LogisticRegression(**logreg_params)

# ADHD sample weights
weights_adhd = ((adhd_male_female['Sex_F'] == 1) & (adhd_male_female['ADHD_Outcome'] == 1)).astype(int) + 1

# Train on full dataset
model_sex_symptom.fit(adhd_male_female[symptom_features], adhd_male_female['Sex_F'])
model_adhd_symptom.fit(adhd_male_female[symptom_features], adhd_male_female['ADHD_Outcome'], sample_weight=weights_adhd)

# ----------------- Predict on Test Set -----------------

# Use model to predict on test set
test_pred_sex = model_sex_symptom.predict(test[symptom_features])
test_pred_adhd = model_adhd_symptom.predict(test[symptom_features])

# If you want fMRI-based predictions (optional / if required):
# model_fmri = LogisticRegressionCV(...same as before...)
# model_fmri.fit(unclear_no_adhd[fmri_features], unclear_no_adhd['Sex_F'])
# test_pred_sex_fmri = model_fmri.predict(test[fmri_features])  # Alternative sex prediction

# ---------------- Submission File ----------------

submission = pd.DataFrame({
    "Id": test.index,
    "Sex_F": test_pred_sex,
    "ADHD_Outcome": test_pred_adhd
})

submission.to_csv("../submissionBasicFeatEng.csv", index=False)
print("Submission file saved as 'submissionBasicFeatEng.csv'")


Submission file saved as 'submissionBasicFeatEng.csv'


In [ ]:
def inference(testdata, model_adhd, model_sex):
    # Fit ADHD model
    model_adhd.fit(train.drop(columns=["ADHD_Outcome", "Sex_F"]), train["ADHD_Outcome"])
    
    # Fit Sex model
    model_sex.fit(train.drop(columns=["ADHD_Outcome", "Sex_F"]), train["Sex_F"])
    
    # Predict
    adhd_pred = model_adhd.predict(test)
    sex_pred = model_sex.predict(test)

    # Prepare submission
    sub['ADHD_Outcome'] = adhd_pred
    sub['Sex_F'] = sex_pred
    sub.to_csv(f'../submission{score}.csv', index=False)



model_adhd = LogisticRegressionCV(
    penalty="l2",
    Cs=10,
    cv=5,
    fit_intercept=True,
    scoring="f1",
    solver="saga",
    random_state=SEED,
    class_weight="balanced",
    max_iter=1000
)

model_sex = LogisticRegressionCV(
    penalty="l2",
    Cs=10,
    cv=5,
    fit_intercept=True,
    scoring="f1",
    solver="saga",
    random_state=SEED,
    class_weight="balanced",
    max_iter=1000
)

inference(test, model_adhd, model_sex)

'''
Code ran in 138m 58.8s
'''

# Trying